In [1]:
%%writefile openmp_array_processing.cpp
#include <iostream>
#include <vector>
#include <omp.h>

using namespace std;

int main() {
    const int N = 1'000'000;
    vector<double> data(N);

    // Инициализация массива
    for (int i = 0; i < N; i++) {
        data[i] = 1.0;
    }

    // Замер времени выполнения
    double startTime = omp_get_wtime();

    // Параллельная обработка массива с использованием OpenMP
    #pragma omp parallel for
    for (int i = 0; i < N; i++) {
        data[i] *= 2.0;
    }

    double endTime = omp_get_wtime();

    // Вывод времени выполнения
    cout << "Время выполнения на CPU (OpenMP): "
         << (endTime - startTime) << " секунд" << endl;

    // Проверка корректности
    cout << "Первые 5 элементов массива: ";
    for (int i = 0; i < 5; i++) {
        cout << data[i] << " ";
    }
    cout << endl;

    return 0;
}


Writing openmp_array_processing.cpp


In [2]:
!g++ -fopenmp openmp_array_processing.cpp -O2 -o openmp_cpu
!./openmp_cpu


Время выполнения на CPU (OpenMP): 0.000490948 секунд
Первые 5 элементов массива: 2 2 2 2 2 


In [3]:
%%writefile cuda_array_processing.cu
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

// Размер массива
const int N = 1'000'000;
const int BLOCK_SIZE = 256;

// CUDA-ядро для поэлементной обработки массива
__global__ void processKernel(float* data, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < n) {
        data[idx] *= 2.0f;
    }
}

int main() {
    // Выделение и инициализация массива на CPU
    float* h_data = new float[N];
    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
    }

    // Выделение памяти на GPU
    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));

    // Копирование данных CPU -> GPU
    cudaMemcpy(d_data, h_data,
               N * sizeof(float),
               cudaMemcpyHostToDevice);

    int gridSize = (N + BLOCK_SIZE - 1) / BLOCK_SIZE;

    // Создание событий для замера времени
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    // Запуск CUDA-ядра и замер времени
    cudaEventRecord(start);
    processKernel<<<gridSize, BLOCK_SIZE>>>(d_data, N);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float gpuTime = 0.0f;
    cudaEventElapsedTime(&gpuTime, start, stop);

    // Копирование данных GPU -> CPU
    cudaMemcpy(h_data, d_data,
               N * sizeof(float),
               cudaMemcpyDeviceToHost);

    // Вывод времени выполнения
    cout << "Время выполнения на GPU (CUDA): "
         << gpuTime << " мс" << endl;

    // Проверка корректности
    cout << "Первые 5 элементов массива: ";
    for (int i = 0; i < 5; i++) {
        cout << h_data[i] << " ";
    }
    cout << endl;

    // Освобождение памяти
    delete[] h_data;
    cudaFree(d_data);

    return 0;
}


Writing cuda_array_processing.cu


In [4]:
!nvcc -arch=sm_75 cuda_array_processing.cu -o cuda_gpu
!./cuda_gpu


Время выполнения на GPU (CUDA): 0.137888 мс
Первые 5 элементов массива: 2 2 2 2 2 


In [5]:
%%writefile hybrid_processing.cu
#include <iostream>
#include <cuda_runtime.h>
#include <omp.h>

using namespace std;

const int N = 1'000'000;
const int HALF = N / 2;
const int BLOCK_SIZE = 256;

// CUDA-ядро для обработки части массива на GPU
__global__ void gpuKernel(float* data, int offset, int n) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < n) {
        data[offset + idx] *= 2.0f;
    }
}

int main() {
    // Выделение и инициализация массива на CPU
    float* h_data = new float[N];
    for (int i = 0; i < N; i++) {
        h_data[i] = 1.0f;
    }

    // Выделение памяти на GPU
    float* d_data;
    cudaMalloc(&d_data, N * sizeof(float));

    // Копирование всего массива на GPU
    cudaMemcpy(d_data, h_data,
               N * sizeof(float),
               cudaMemcpyHostToDevice);

    int gpuSize = N - HALF;
    int gridSize = (gpuSize + BLOCK_SIZE - 1) / BLOCK_SIZE;

    double startTime = omp_get_wtime();

    // Параллельное выполнение CPU и GPU
    #pragma omp parallel sections
    {
        // CPU часть
        #pragma omp section
        {
            #pragma omp parallel for
            for (int i = 0; i < HALF; i++) {
                h_data[i] *= 2.0f;
            }
        }

        // GPU часть
        #pragma omp section
        {
            gpuKernel<<<gridSize, BLOCK_SIZE>>>(d_data, HALF, gpuSize);
            cudaDeviceSynchronize();
        }
    }

    double endTime = omp_get_wtime();

    // Копирование обработанных данных обратно на CPU
    cudaMemcpy(h_data + HALF,
               d_data + HALF,
               gpuSize * sizeof(float),
               cudaMemcpyDeviceToHost);

    cout << "Общее время гибридной обработки: "
         << (endTime - startTime) << " секунд" << endl;

    // Проверка корректности
    cout << "Первые 5 элементов: ";
    for (int i = 0; i < 5; i++) {
        cout << h_data[i] << " ";
    }
    cout << endl;

    cout << "Последние 5 элементов: ";
    for (int i = N - 5; i < N; i++) {
        cout << h_data[i] << " ";
    }
    cout << endl;

    delete[] h_data;
    cudaFree(d_data);

    return 0;
}


Writing hybrid_processing.cu


In [6]:
!nvcc -arch=sm_75 -Xcompiler -fopenmp hybrid_processing.cu -o hybrid
!./hybrid


Общее время гибридной обработки: 0.00143098 секунд
Первые 5 элементов: 2 2 2 2 2 
Последние 5 элементов: 2 2 2 2 2 


In [ ]:
%%writefile